# MICRO-452 Project Report - Groupe 

**Authors:** David Croce                   

**Supervisors:** Prof. Francesco Mondada                    

**Due date:** 07.12.2023          

## Table of Contents
<ol>
<li>Introduction</li>
<li>Vision</li>
<li>Global Navigation</li>
<li>Local Navigation</li>
<li>Motion Control</li>
<li>Kalman Filter</li>
<li>Main</li>
<li>Conclusion</li>
</ol>

## 1. Robot and Map classes


### Now we import the necessary modules 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time 
import sys 

sys.path.append('.\src')

from main import *



Starting camera...
Running threads...


In [1]:
from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node 403e6256-30ac-42f4-bb1b-1c2b7dabf7d5

### Here, our only output is the speed of each motor. Our infinite loop updates the values via the update_main() function which returns the speed of the motors.

In [3]:

def motors(left, right):
    return {
        "motor.left.target": [left],
        "motor.right.target": [right],
    }

def get_data():
    thymio_data = np.array({"left_speed":node["motor.left.speed"],
                        "right_speed":node["motor.right.speed"]})

    return thymio_data

start_time = time.time()
while True:
    

    await node.wait_for_variables({"motor.left.speed", "motor.right.speed"})
    speed_left = node.v.motor.left.speed/2
    speed_right = node.v.motor.right.speed/2
    
    avg_speed = (speed_left+speed_right)/2

    angular_speed = (speed_right-speed_left)/80
    #print(speed_left, "   ", speed_right)

    v = update_main(avg_speed, angular_speed)
    node.send_set_variables(v)
    await client.sleep(0.1)


    



Execution time:  0.10328173637390137
Execution time:  0.11207842826843262
Execution time:  0.10775375366210938
Execution time:  0.10971188545227051
Execution time:  0.10737800598144531
Execution time:  0.10862350463867188
Execution time:  0.10871195793151855
Execution time:  0.10759711265563965
Execution time:  0.10758662223815918
Execution time:  0.10655689239501953
Execution time:  0.10705137252807617
Execution time:  0.10687041282653809
Execution time:  0.10585570335388184
Execution time:  0.10805988311767578
Execution time:  0.10723257064819336
Execution time:  0.10940194129943848
Execution time:  0.10978126525878906
Execution time:  0.10854697227478027
Execution time:  0.10885262489318848
Execution time:  0.10786747932434082
Execution time:  0.1087796688079834
Execution time:  0.10811924934387207
Execution time:  0.10781431198120117
Execution time:  0.1082916259765625
Execution time:  0.10831260681152344
Execution time:  0.10711932182312012
Execution time:  0.10692858695983887
Exe

In [2]:
def motors(left, right):
    return {
        "motor.left.target": [left],
        "motor.right.target": [right],
    }

node.send_set_variables(motors(0,0))

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time 
import sys 

from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()



def get_data():
    thymio_data = np.array({"left_speed":node["motor.left.speed"],
                        "right_speed":node["motor.right.speed"]})

    return thymio_data

v = {"motor.left.target": [100],
    "motor.right.target": [100],}


while True:
    await node.wait_for_variables({"motor.left.speed", "motor.right.speed"})
    print(node.v.motor.left.speed, "   ", node.v.motor.right.speed)
    
    node.send_set_variables(v)

    await client.sleep(0.1)
 

-83     108
-83     108
0     37
0     37
83     89
83     89
98     101
98     101
99     98
99     98
99     98
100     95
100     95
101     101
101     101
98     101
98     101
98     101
96     100
96     100
103     92
103     92
106     101
106     101
102     103
102     103
105     103
105     103
100     99
100     99
100     99
97     100
97     100
83     96
83     96
92     100
92     100
92     100
101     102
101     102
101     102
99     103
101     101
101     101
101     101
101     93
101     93
99     100
99     100
104     101
104     101
96     102
92     101
92     101
92     101
64     100
64     100
-3     102
-3     102
-3     102
-24     101
-26     96
-26     96
-26     96
-26     101
-3     95
-3     95
-3     95
-6     103
-6     103
104     102
104     102
105     103
105     103
102     101
102     101
137     100
137     100
105     91
105     91
105     91
100     99
100     99
100     98
100     98
48     98
48     98
0     93
0     93
0     93
-28 

CancelledError: 

In [ ]:
import matplotlib.pyplot as plt
from collections import deque
import numpy as np

def is_valid(x, y, grid, robot_size):
    n = len(grid)
    for i in range(max(0, x - robot_size + 1), min(n, x + robot_size)):
        for j in range(max(0, y - robot_size + 1), min(n, y + robot_size)):
            if not (0 <= i < n and 0 <= j < n) or grid[i][j] == 1:
                return False
    return True

def find_shortest_path(grid, start, goal, robot_size):
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]
    queue = deque([(start, [])])
    visited = set()
    n = len(grid)

    while queue:
        (x, y), path = queue.popleft()
        if (x, y) == goal:
            return path + [(x, y)]

        if (x, y) not in visited and 0 <= x < n and 0 <= y < n:
            visited.add((x, y))
            for dx, dy in directions:
                new_x, new_y = x + dx, y + dy
                if is_valid(new_x, new_y, grid, robot_size) and 0 <= new_x < n and 0 <= new_y < n:
                    queue.append(((new_x, new_y), path + [(x, y)]))

    return None


def plot_grid(grid, path, start, goal):
    n = len(grid)
    plt.figure(figsize=(8, 8))
    plt.imshow(grid, cmap='Greys', interpolation='nearest')

    if path:
        path_x, path_y = zip(*path)
        plt.plot(path_y, path_x, marker='o', color='red')

    plt.plot(start[1], start[0], marker='o', color='green', markersize=10)
    plt.plot(goal[1], goal[0], marker='o', color='blue', markersize=10)

    plt.xlim(-0.5, n - 0.5)
    plt.ylim(n - 0.5, -0.5)
    plt.xticks(np.arange(-0.5, n, 1))
    plt.yticks(np.arange(-0.5, n, 1))
    plt.gca().invert_yaxis()
    plt.grid(visible=True, color='black', linestyle='-', linewidth=0.5)
    plt.show()

def find_direction_changes(path):
    if len(path) <= 2:
        return path

    direction_changes = [path[0]]

    for i in range(2, len(path)):
        dx1 = path[i - 1][0] - path[i - 2][0]
        dy1 = path[i - 1][1] - path[i - 2][1]
        dx2 = path[i][0] - path[i - 1][0]
        dy2 = path[i][1] - path[i - 1][1]

        if (dx1, dy1) != (dx2, dy2):
            direction_changes.append(path[i - 1])

    direction_changes.append(path[-1])

    return direction_changes

# Example grid with obstacles

grid = np.array([
    [0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0],
    [0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0],
    [0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0],
    [0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0],
    [1 ,1 ,1 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,1 ,1],
    [1 ,1 ,1 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,1 ,1],
    [1 ,1 ,1 ,0 ,0 ,0 ,1 ,1 ,1 ,1 ,1 ,1 ,0 ,1 ,1],
    [0 ,0 ,0 ,0 ,0 ,0 ,1 ,1 ,1 ,1 ,1 ,1 ,0 ,1 ,1],
    [0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,1 ,1],
    [0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,1 ,1],
    [0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0],
    [0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0],
    [0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0],
    [0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0],
    [0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0]
    ])

#filp rows of grid
grid = np.flip(grid,0)

start_index = (2, 1)  # Replace with the index of your robot
goal_index = (12, 13)  # Replace with the index of your goal
robot_size = 2  # Replace with the size of your robot

path = find_shortest_path(grid, start_index, goal_index, robot_size)
if path:
    direction_changes_path = find_direction_changes(path)
    print("Path with direction changes:", direction_changes_path)
    plot_grid(grid, direction_changes_path, start_index, goal_index)
    # plot_grid(grid, path, start_index, goal_index)
else:
    print("No path found.")
